<span style="float:left;">Licence CC BY-NC-ND</span><span style="float:right;">Thierry Parmentelat &amp; Arnaud Legout&nbsp;<img src="media/both-logos-small-alpha.png" style="display:inline"></span><br/>

# boucle d'événements `asyncio`

In [ ]:
import asyncio

# utilitaire

fonctions synchrones (traditionnelles)

In [ ]:
from asynchelpers import start_timer, show_timer

In [ ]:
import time

start_timer()
time.sleep(1)
show_timer('un message')

# déjà vu

`loop.run_until_complete()`

* exactement un argument

```
asyncio.get_event_loop().run_until_complete(
    asyncio.gather(coro1, coro2, ...)
))
```

# ajout de traitements

`asyncio.ensure_future(coro)`

* exactement un argument
* ajoute une coroutine dans la boucle
* **avant** ou **après** le lancement de la boucle

`loop.run_forever()`

* sans argument
* travaille sur le contenu courant de la boucle
* suppose l'utilisation de `ensure_future()`

# *fork*

![figure fork](w8-s6-av-fig1.png)

In [ ]:
async def c1():
    show_timer(">>> c1")
    await asyncio.sleep(1)
    show_timer("forking")
    # fork
    asyncio.ensure_future(c2())
    await asyncio.sleep(1)
    show_timer("<<< c1")

In [ ]:
# sera forkée par c1() après une seconde

async def c2():
    show_timer(">>> c2")
    await asyncio.sleep(2)
    show_timer("<<< c2")
    
    

In [ ]:
asyncio.ensure_future(c1())

In [ ]:
start_timer()

# interrompre après 2s
try:
    asyncio.get_event_loop().run_forever()
except KeyboardInterrupt:
    print("bye")

# réinitialisation de la boucle

In [ ]:
asyncio.set_event_loop(asyncio.new_event_loop())

*je vous demande de l'admettre pour l'instant*

# `loop.stop()`

In [ ]:
async def c1_stop():
    show_timer(">>> c1")
    await asyncio.sleep(1)
    show_timer("forking")
    # fork
    asyncio.ensure_future(c2_stop())
    await asyncio.sleep(1)
    show_timer("<<< c1")

In [ ]:
# sera forkée par c1() après une seconde

async def c2_stop():
    show_timer(">>> c2")
    await asyncio.sleep(2)
    show_timer("<<< c2")
    # attention c'est une méthode bloquante
    asyncio.get_event_loop().stop()

In [ ]:
asyncio.ensure_future(c1_stop())

In [ ]:
start_timer()

# s'arrête tout seul
try:
    asyncio.get_event_loop().run_forever()
except KeyboardInterrupt:
    print("bye")
    
print("done")

# `run_until_complete` sur une boucle non-vide 

In [ ]:
asyncio.set_event_loop(asyncio.new_event_loop())

In [ ]:
# on simule un job asynchrone de durée duration
async def job(name, duration):
    show_timer(f">>> {name}")
    await asyncio.sleep(duration)
    show_timer(f"<<< {name}")    

In [ ]:
async def short():
    await job("short", 1)

In [ ]:
async def long():
    await job("long", 2)

In [ ]:
# on remplit la boucle
asyncio.ensure_future(long())

In [ ]:
# et on appelle run_until_complete
start_timer()

asyncio.get_event_loop().run_until_complete(short())
print("done")

In [ ]:
# il reste des choses à faire dans la boucle
start_timer()

# interrompre après 1s
try:
    asyncio.get_event_loop().run_forever()
except KeyboardInterrupt:
    print("bye")

# `run_until_complete` vs `run_forever`

* `run_until_complete`

   * prend exactement un argument
   * retourne la valeur
   * insérer un fragment asynchrone au milieu d'un code synchrone


* `run_forever`
  * ne prend pas d'argument
  * ne retourne pas (sauf en cas de `stop()`)
  * orienté traitement massivement asynchrone

# `get_event_loop()`

* modèle mental
  * 1 thread = 1 boucle

* `get_event_loop()` 
  * boucle par défaut du thread courant
  * utile pour référencer "la bonne boucle" - voir e.g. le code de `c2_stop()`
  * inutile de passer une instance de boucle
  * ne *crée pas* de boucle en dehors du thread principal  

# `new_event_loop()` et `set_event_loop()`

* `new_event_loop()`
  * crée une nouvelle boucle
  * nécessaire dans un thread supplémentaire

* set_event_loop(loop)
  * installe cet objet comme boucle par défaut

# résumé

* `ensure_future()`

* `run_until_complete()` et `run_forever()`

* `get_event_loop()` 
  * accéde à la boucle (du thread) courant(e)
  * `new_event_loop()` et `set_event_loop()`

* `loop.stop()`